In [7]:
from lib.functions0 import *
from lib.utility import *
import numpy as np
from scipy.linalg import expm
from qiskit.utils.mitigation.fitters import CompleteMeasFitter
from qiskit.ignis.mitigation.measurement import  complete_meas_cal
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt
#%load_ext autoreload
#%autoreload 2
import warnings
warnings.filterwarnings('ignore')
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community',
                            group='ibmquantumawards', 
                            project='open-science-22')

backend_sim_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_real_jakarta = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')
import qiskit
qiskit.utils.mitigation.fitters.__file__


Passi = 200
n_steps=100
#initial_state={"110": 1}
shots = 8000
backend = backend_sim_jakarta

X = np.array([[0,1],[1,0]])  #defining the pauli matrices
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
H = np.array([[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]])
Id = np.eye(2)

# defining the hamiltonian divided in: 
#       - H1: first two qubits interactions.
#       - H2: second two qubits interactions.

H1 = np.kron(X, np.kron(X,Id)) + np.kron(Y, np.kron(Y,Id)) + np.kron(Z, np.kron(Z,Id)) 
H2 = np.kron(Id, np.kron(X,X)) + np.kron(Id, np.kron(Y,Y)) + np.kron(Id, np.kron(Z,Z)) 

Had3=np.kron(H, np.kron(H,H))

fids_list = np.zeros([Passi, 4])
time_range = np.linspace(0.001,np.pi, Passi)

for iii, time in enumerate(time_range):
    # building numerically the trotter step matrix, and the whole operator (trotter step)^n_steps.

    trotter_step_matrix_= expm(-time/n_steps*H1*1j).dot(expm(-time/n_steps*H2*1j))
    trotterized = np.linalg.matrix_power(trotter_step_matrix_, n_steps)
    Matrix(trotterized).n(3, chop=True)
    #Matrix(Had3)
    qr=QuantumRegister(3, name="q")
    B_qc=QuantumCircuit(qr, name="B")
    B_qc.x(qr[2])
    B_qc.cx(qr[1],qr[0])
    B_qc.cx(qr[2],qr[1])
    B_qc.cx(qr[1],qr[0])
    B_qc.x([qr[0],qr[1],qr[2]])
    B_qc.append(Toffoli_gate,[qr[0],qr[1],qr[2]])
    B_qc.x([qr[0],qr[1]])

    B_qc.draw(output="mpl")
    B=matrix_from_circuit(B_qc)
    B
    transpile(B_qc, basis_gates=["cx", "x", "rz", "sx"]).draw(output="mpl")
    #for _ in range(n_steps): # here is computed the evolution operator numerically, with n_steps trotter steps.
    #
    #    numeric_evolution_matrix=(numeric_evolution_matrix*trotter_step_matrix_(2*time/n_steps)).evalf(precision)


    M_N = B*trotterized*B.H
    M_N = Matrix([M_N[0:4],M_N[8:12],M_N[16:20],M_N[24:28]])
    M_N.evalf(5)
    qc=QuantumCircuit(2, name="$M^N$")
    qc.unitary(M_N,[0,1])    
    M_N_qc=transpile(qc,basis_gates=['cx','x','sx','rz']) 

    M_N_qc.draw(output="mpl")
    initial_state="000"

    qr_U = QuantumRegister(3, name="q")
    qc_U = QuantumCircuit(qr_U, name="evo")

    ### preparing the initial state

    l=0
    for k in [2,1,0]:
        if initial_state[l]=='1':
            qc_U.x(qr_U[k])
        l+=1

    ### appending the evolution

    qc_U.append(M_N_qc,[qr_U[0],qr_U[1]])
    qc_U.append(B_qc.inverse(),[qr_U[0],qr_U[1],qr_U[2]])

    qc_U.draw(output="mpl")
    U_ideal=matrix_from_circuit(qc_U, type="numpy")
    Matrix(U_ideal)
    qr_evo = QuantumRegister(3, name="q")
    qc_evo = QuantumCircuit(qr_evo, name="U")

    qc_evo.x([qr_evo[1],qr_evo[2]])
    qc_evo.append(qc_U, qr_evo)

    qcs_tomo = state_tomography_circuits(qc_evo, qr_evo)
    qcs_tomo[3].draw(output="mpl")
    qr_cal = QuantumRegister(3)
    cal_circ, state_labels = complete_meas_cal(qubit_list=[0,1,2], qr=qr_cal, circlabel='mcal')
    cal_circ[0].draw()
    state_labels
    calib_circuits_identity = []
    calib_circuits_itself = []

    for i in state_labels:

        cr_cal_itself = ClassicalRegister(3)
        qr_cal_itself = QuantumRegister(3)
        qc_cal_itself = QuantumCircuit(qr_cal_itself, cr_cal_itself, name=f"mcalcal_{i}")

        cr_cal_id = ClassicalRegister(3)
        qr_cal_id = QuantumRegister(3)
        qc_cal_id = QuantumCircuit(qr_cal_id, cr_cal_id, name=f"mcalcal_{i}")

        qc_cal_id.x(qr_cal_id)
        qc_cal_id.append(qc_U, qr_cal_id)

        for k in range(3):
            if i[::-1][k] == "1":
                qc_cal_itself.x(qr_cal_itself[k])
            else:
                qc_cal_id.x(qr_cal_id[k])
            
        qc_cal_itself.append(qc_U, qr_cal_itself)
        
        qc_cal_id.measure(qr_cal_id, cr_cal_id)
        qc_cal_itself.measure(qr_cal_itself, cr_cal_itself)

        calib_circuits_identity.append(qc_cal_id)
        calib_circuits_itself.append(qc_cal_itself)
    calib_circuits_identity[1].draw()
    calib_circuits_itself[1].draw()
    job_tomo=execute(qcs_tomo, backend, shots=shots, initial_layout=[1,3,5])

    job_cal_our_identity=execute(calib_circuits_identity, backend = backend, shots=shots, initial_layout=[1,3,5])

    job_cal_our_itself=execute(calib_circuits_itself, backend = backend, shots=shots, initial_layout=[1,3,5])

    job_cal_qiskit=execute(cal_circ, backend, shots=shots, initial_layout=[1,3,5])
    meas_fitter_our_identity = CompleteMeasFitter(job_cal_our_identity.result(), state_labels=state_labels)
    meas_fitter_our_itself = CompleteMeasFitter(job_cal_our_itself.result(), state_labels=state_labels)
    meas_fitter_qiskit = CompleteMeasFitter(job_cal_qiskit.result(), state_labels=state_labels)
    Matrix(meas_fitter_qiskit.cal_matrix)
    Matrix(meas_fitter_our_identity.cal_matrix)
    U_tilde_identity=meas_fitter_our_identity.cal_matrix
    U_tilde_itself=meas_fitter_our_itself.cal_matrix
    U_tilde_qiskit=meas_fitter_qiskit.cal_matrix
    def matrix_from_cirquit(qc, phase=0, type="sympy"):

        backend = Aer.get_backend('unitary_simulator')
        job = execute(qc, backend, shots=32000)
        result = job.result()
        A=result.get_unitary(qc, decimals=10)*np.exp(1j*phase)
        if type=="sympy":
            return Matrix(A)
        else:
            return A
    qr_basi = QuantumRegister(3)
    qc_basi = QuantumCircuit(qr_basi)

    qcs_basis = state_tomography_circuits(qc_basi, qr_basi)

    qcs_basis[0].remove_final_measurements()

    qcs_basis[16].draw()
    meas_fitter_qiskit.cal_matrix
    C_matrices_itself = []
    C_matrices_identity = []

    U_ideal_inv = np.linalg.inv(U_ideal)
    U_ideal_inv_abs = np.abs(U_ideal_inv)**2

    U_ideal_abs = np.abs(U_ideal)**2
    #print(type(U_ideal_abs))
    #U_ideal_abs = np.asarray(Matrix(U_ideal_abs).n(10, chop=True), dtype='float64')
    #print(type(U_ideal_abs))
    U_ideal_abs_inv = np.linalg.inv(U_ideal_abs)

    C_itself = np.dot(U_tilde_itself, U_ideal_abs_inv)
    C_identity = U_tilde_identity
    
    for base in qcs_basis:
        
        base.remove_final_measurements()

        base_matrix_amplitudes = matrix_from_cirquit(base, type="numpy")
        base_matrix_amplitudes_inverse = np.linalg.inv(base_matrix_amplitudes)

        base_matrix = np.abs(base_matrix_amplitudes)**2
        base_matrix_inverse = np.abs(base_matrix_amplitudes_inverse)**2

        #C_aus_itself = np.linalg.multi_dot([base_matrix, C_itself,  base_matrix_inverse])
        C_aus_itself = np.linalg.multi_dot([U_tilde_qiskit, base_matrix, np.linalg.inv(U_tilde_qiskit), C_itself,  base_matrix_inverse])
        C_aus_itself = np.linalg.multi_dot([U_tilde_qiskit, base_matrix, np.linalg.inv(U_tilde_qiskit), C_itself,  np.transpose(base_matrix)])


        C_aus_identity = np.linalg.multi_dot([U_tilde_qiskit, base_matrix, np.linalg.inv(U_tilde_qiskit), C_identity,  base_matrix_inverse])
        #C_aus_identity = np.linalg.multi_dot([base_matrix, C_identity, base_matrix_inverse])


        C_matrices_identity.append(np.asarray(C_aus_identity))
        C_matrices_itself.append(np.asarray(C_aus_itself))
    from copy import deepcopy

    meas_fitters_identity = []
    meas_fitters_itself = []

    for C_new in C_matrices_identity:
        meas_fitter_our_aus = deepcopy(meas_fitter_our_identity)
        meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

        meas_fitters_identity.append(meas_fitter_our_aus)

    for C_new in C_matrices_itself:
        meas_fitter_our_aus = deepcopy(meas_fitter_our_itself)
        meas_fitter_our_aus._tens_fitt.cal_matrices[0]=C_new

        meas_fitters_itself.append(meas_fitter_our_aus)
    target_state = (One^One^Zero).to_matrix()
    #target_state = (Zero^One^One).to_matrix()


    fids=np.zeros(4)

    raw_res=deepcopy(job_tomo.result())
    qiskit_res=deepcopy(raw_res)
    identity_res=deepcopy(raw_res)
    itself_res=deepcopy(raw_res)
    raw_res.get_counts(-1)
    for i in range(27):

        old_counts=raw_res.get_counts(i)
        new_counts_qiskit = meas_fitter_qiskit.filter.apply(old_counts, method="least_squares")
        qiskit_res.results[i].data.counts = new_counts_qiskit

        new_counts_id = meas_fitters_identity[i].filter.apply(old_counts, method="least_squares")
        identity_res.results[i].data.counts = new_counts_id

        new_counts_it = meas_fitters_itself[i].filter.apply(old_counts, method="least_squares")
        itself_res.results[i].data.counts = new_counts_it
    qiskit_res.get_counts(-1)
    It_comp=occurrences_to_vector(itself_res.get_counts(-1))/(shots)
    It_H=np.dot(Had3,It_comp)
    Matrix(It_H)

    itself_res.get_counts(0)
    itself_res.get_counts(-1)
    identity_res.get_counts(-1)
    fids[0] = fidelity_count(raw_res, qcs_tomo, target_state)
    fids[1] = fidelity_count(qiskit_res, qcs_tomo, target_state)
    fids[2] = fidelity_count(identity_res, qcs_tomo, target_state)
    fids[3] = fidelity_count(itself_res, qcs_tomo, target_state)

    fids_list[iii, :] = fids
    print(time)



ibmqfactory.load_account:WARNING:2022-04-11 23:57:40,222: Credentials are already in use. The existing account in the session will be replaced.


0.001
0.016781872631104488
0.032563745262208975
0.04834561789331346
0.06412749052441795
0.07990936315552244
0.09569123578662692
0.1114731084177314
0.1272549810488359
0.14303685367994037
0.15881872631104488
0.17460059894214935
0.19038247157325383
0.20616434420435834
0.2219462168354628
0.23772808946656732
0.2535099620976718
0.26929183472877627
0.28507370735988075
0.3008555799909853
0.31663745262208975
0.33241932525319423
0.3482011978842987
0.3639830705154032
0.37976494314650766
0.3955468157776122
0.41132868840871667
0.42711056103982115
0.4428924336709256
0.4586743063020301
0.47445617893313463
0.4902380515642391
0.5060199241953436
0.5218017968264481
0.5375836694575525
0.5533655420886571
0.5691474147197615
0.584929287350866
0.6007111599819706
0.616493032613075
0.6322749052441795
0.6480567778752839
0.6638386505063885
0.679620523137493
0.6954023957685974
0.711184268399702
0.7269661410308064
0.7427480136619109
0.7585298862930153
0.7743117589241199
0.7900936315552244
0.8058755041863288
0.82165

In [8]:


occurrences_to_vector(identity_res.get_counts(-1))
# Confronto caso ideale
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})  # enlarge matplotlib fonts

# Import qubit states Zero (|0>) and One (|1>), and Pauli operators (X, Y, Z)
from qiskit.opflow import Zero, One, I, X, Y, Z

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Returns the matrix representation of the XXX Heisenberg model for 3 spin-1/2 particles in a line
def H_heis3():
    # Interactions (I is the identity matrix; X, Y, and Z are Pauli matricies; ^ is a tensor product)
    XXs = (I^X^X) + (X^X^I)
    YYs = (I^Y^Y) + (Y^Y^I)
    ZZs = (I^Z^Z) + (Z^Z^I)
    
    # Sum interactions
    H = XXs + YYs + ZZs
    
    # Return Hamiltonian
    return H

# Returns the matrix representation of U_heis3(t) for a given time t assuming an XXX Heisenberg Hamiltonian for 3 spins-1/2 particles in a line
def U_heis3(t):
    # Compute XXX Hamiltonian for 3 spins in a line
    H = H_heis3()
    
    # Return the exponential of -i multipled by time t multipled by the 3 spin XXX Heisenberg Hamilonian 
    return (t * H).exp_i()
initial_state = One^One^Zero

#((initial_state @ U_heis3(float(time)) @ initial_state).eval())**2
np.abs((~initial_state @ U_heis3(float(time)) @ initial_state).eval())**2








1.0000000000000004

In [9]:
print(time_range)

[1.00000000e-03 1.67818726e-02 3.25637453e-02 4.83456179e-02
 6.41274905e-02 7.99093632e-02 9.56912358e-02 1.11473108e-01
 1.27254981e-01 1.43036854e-01 1.58818726e-01 1.74600599e-01
 1.90382472e-01 2.06164344e-01 2.21946217e-01 2.37728089e-01
 2.53509962e-01 2.69291835e-01 2.85073707e-01 3.00855580e-01
 3.16637453e-01 3.32419325e-01 3.48201198e-01 3.63983071e-01
 3.79764943e-01 3.95546816e-01 4.11328688e-01 4.27110561e-01
 4.42892434e-01 4.58674306e-01 4.74456179e-01 4.90238052e-01
 5.06019924e-01 5.21801797e-01 5.37583669e-01 5.53365542e-01
 5.69147415e-01 5.84929287e-01 6.00711160e-01 6.16493033e-01
 6.32274905e-01 6.48056778e-01 6.63838651e-01 6.79620523e-01
 6.95402396e-01 7.11184268e-01 7.26966141e-01 7.42748014e-01
 7.58529886e-01 7.74311759e-01 7.90093632e-01 8.05875504e-01
 8.21657377e-01 8.37439249e-01 8.53221122e-01 8.69002995e-01
 8.84784867e-01 9.00566740e-01 9.16348613e-01 9.32130485e-01
 9.47912358e-01 9.63694230e-01 9.79476103e-01 9.95257976e-01
 1.01103985e+00 1.026821

In [10]:
fids_list[1,:]

array([0.82995324, 0.92070646, 0.97322947, 0.98843062])

In [11]:
Passi

200

In [12]:
t_ideal = np.linspace(0,np.pi, 1000)
probs_110 = [np.abs((~initial_state @ U_heis3(float(t)) @ initial_state).eval())**2 for t in t_ideal]

labels = ['Raw', 'Qiskit', 'Identity', 'Ciruit']

t=time_range
plt.figure(figsize=(13,10), dpi=1000)
plt.axvline(np.pi/2)
plt.plot(t_ideal, probs_110)
for i in range(4):
    plt.plot(time_range, fids_list[:,i], linestyle='', marker='*', label=labels[i])

#plt.xlim(np.pi/2-0.0099,np.pi/2+0.0099)
plt.legend()
plt.grid()
plt.xlabel("time")
plt.ylabel("fidelity")
plt.title("number of trotter ")
plt.show()